# Harmony GDAL Adapter (HGA) regression tests

The scope of this Jupyter notebook is to run a suite of regression tests against sample collections for the Harmony GDAL Adapter (HGA). These sample collections are primarily from the Alaska Satellite Facility (ASF), including:

* [ALOS AVNIR OBS ORI](https://cmr.earthdata.nasa.gov/search/concepts/C1808440897-ASF.html)
* [Sentinel 1 Interferograms](https://cmr.earthdata.nasa.gov/search/concepts/C1595422627-ASF.html)
* [UAVSAR POLSAR Pauli](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1244141264-EEDTEST.html)
* [GHRSST Level 4 MUR Global Foundation Sea Surface Temperature Analysis (v4.1)](https://podaac.jpl.nasa.gov/dataset/MUR-JPL-L4-GLOB-v4.1)

In [ ]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'

### Import required packages:

In [ ]:
from datetime import datetime

from harmony import BBox, Client, Collection, Environment, Request

from test_utilities import (check_request_output, clean_test_artefacts,
                            make_request_and_download_result)

### Set up environment dependent variables:

In [ ]:
collection_data = {
    'https://harmony.uat.earthdata.nasa.gov': {
        'env': Environment.UAT,
        'avnir_collection': Collection(id='C1244141281-EEDTEST'),
        'sentinel_collection': Collection(id='C1244141250-EEDTEST'),
        'uavsar_collection': Collection(id='C1244141264-EEDTEST'),
        'mur_collection': Collection(id='C1238621141-POCLOUD'),
    } 
}

environment_information = collection_data.get(harmony_host_url)

if environment_information is not None:
    harmony_client = Client(env=environment_information['env'])

## AVNIR

The tests below will use the ALOS AVNIR OBS ORI collection (for UAT this is mirrored in the EEDTEST CMR environment). There are no reference images provided for this collection, as they would each be large (tens of MB).

### AVNIR test case 1:

This test case combines a bounding box spatial subset across the Equator with a variable subset for bands 1 and 2.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1813212660-ASF',
                Environment.UAT: 'G1244144597-EEDTEST',
                Environment.SIT: 'G1244144597-EEDTEST'}

    avnir_directory_one = 'avnir_test_one'
    
    avnir_request_one = Request(collection=environment_information['avnir_collection'],
                                spatial=BBox(-51.0, -0.05, -50.75, 0.25),
                                variables=['Band1', 'Band2'], format='image/tiff',
                                granule_id=granules[environment_information['env']])

    avnir_output_one = make_request_and_download_result(harmony_client, avnir_request_one,
                                                        avnir_directory_one)


    avnir_variable_base_one = 'ALAV2A237403590-OORIRFU'
    
    avnir_variables_one = {f'IMG-01-{avnir_variable_base_one}_000_band_1',
                           f'IMG-02-{avnir_variable_base_one}_000_band_1'}
    

    expected_avnir_results_one = {'authority': 'EPSG',
                                  'cs': 'Projected',
                                  'gcs': 'WGS 84',
                                  'gcs_epsg': '4326',
                                  'n_bands': 2,
                                  'proj_cs':'WGS 84 / UTM zone 22N',
                                  'proj_epsg': '32622',
                                  'spatial_extent': [-0.05, 0.3, -50.99, -50.75],
                                  'variables': avnir_variables_one,
                                  'width': 3410, 
                                  'height': 3238}
    
    check_request_output(avnir_output_one[0], expected_avnir_results_one)
    clean_test_artefacts(avnir_directory_one)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 2:

This test case combines a bounding box spatial subset across the Prime Meridian with a variable subset of bands 3 and 4.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1817336128-ASF',
                Environment.UAT: 'G1244144599-EEDTEST',
                Environment.SIT: 'G1244144599-EEDTEST'}

    avnir_directory_two = 'avnir_test_two'
    
    avnir_request_two = Request(collection=environment_information['avnir_collection'],
                                spatial=BBox(-0.15, 14.8, 0.15, 14.9),
                                variables=['Band3', 'Band4'], format='image/tiff',
                                granule_id=granules[environment_information['env']])

    avnir_output_two = make_request_and_download_result(harmony_client, avnir_request_two,
                                                        avnir_directory_two)

    avnir_variable_base_two = 'ALAV2A264953300-OORIRFU'

    avnir_variables_two = {f'IMG-03-{avnir_variable_base_two}_000_band_1',
                           f'IMG-04-{avnir_variable_base_two}_000_band_1'}

    expected_avnir_results_two = {'authority': 'EPSG',
                                  'cs': 'Projected',
                                  'gcs': 'WGS 84',
                                  'gcs_epsg': '4326',
                                  'n_bands': 2,
                                  'proj_cs':'WGS 84 / UTM zone 30N',
                                  'proj_epsg': '32630',
                                  'spatial_extent': [14.74, 14.96, -0.14, 0.14],
                                  'variables': avnir_variables_two,
                                  'width': 3392,
                                  'height': 1762}

    check_request_output(avnir_output_two[0], expected_avnir_results_two)
    clean_test_artefacts(avnir_directory_two)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 3:

This test combines a bounding box spatial subset across the anti-meridian with a variable subset for bands 1 and 2.  

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1835935580-ASF',
                Environment.UAT: 'G1244144598-EEDTEST',
                Environment.SIT: 'G1244144598-EEDTEST'}

    avnir_directory_three = 'avnir_test_three'
    
    avnir_request_three = Request(collection=environment_information['avnir_collection'],
                                  spatial=BBox(179.75, 66.5, -179.85, 66.75),
                                  variables=['Band1', 'Band2'], format='image/tiff',
                                  granule_id=granules[environment_information['env']])

    avnir_output_three = make_request_and_download_result(harmony_client, avnir_request_three,
                                                          avnir_directory_three)

    avnir_variable_base_three = 'ALAV2A239652240-OORIRFU'

    avnir_variables_three = {f'IMG-01-{avnir_variable_base_three}_002_band_1',
                             f'IMG-02-{avnir_variable_base_three}_002_band_1'}

    expected_avnir_results_three = {'authority': 'EPSG',
                                    'cs': 'Projected',
                                    'gcs': 'WGS 84',
                                    'gcs_epsg': '4326',
                                    'n_bands': 2,
                                    'proj_cs':'WGS 84 / UTM zone 1N',
                                    'proj_epsg': '32601',
                                    'spatial_extent': [66.44, 66.81, 179.85, -179.95],
                                    'variables': avnir_variables_three,
                                    'width': 2694,
                                    'height': 3250}
    
    check_request_output(avnir_output_three[0], expected_avnir_results_three)
    clean_test_artefacts(avnir_directory_three)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 4:

This test case combines a bounding box spatial subset in the Northern Hemisphere with a variable subset requesting only band 4.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 1 output band.

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1849288272-ASF',
                Environment.UAT: 'G1244144595-EEDTEST',
                Environment.SIT: 'G1244144595-EEDTEST'}

    avnir_directory_four = 'avnir_test_four'
    
    avnir_request_four = Request(collection=environment_information['avnir_collection'],
                                 spatial=BBox(-99.15, 36.9, -98.85, 37.1),
                                 variables=['Band4'], format='image/tiff',
                                 granule_id=granules[environment_information['env']])

    avnir_output_four = make_request_and_download_result(harmony_client, avnir_request_four,
                                                         avnir_directory_four)

    avnir_variable_base_four = 'ALAV2A278852850-OORIRFU'
    avnir_variables_four = {f'IMG-04-{avnir_variable_base_four}_000_band_1'}

    expected_avnir_results_four = {'authority': 'EPSG',
                                   'cs': 'Projected',
                                   'gcs': 'WGS 84',
                                   'gcs_epsg': '4326',
                                   'n_bands': 1,
                                   'proj_cs':'WGS 84 / UTM zone 14N',
                                   'proj_epsg': '32614',
                                   'spatial_extent': [36.85, 37.15, -99.13, -98.87],
                                   'variables': avnir_variables_four,
                                   'width': 3108,
                                   'height': 2774}
    
    check_request_output(avnir_output_four[0], expected_avnir_results_four)
    clean_test_artefacts(avnir_directory_four)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 5:

This test case requests a bounding box spatial subsest in the Northern Hemisphere. No variable subset is specified, so the request will retrieve all variables.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving all variables and expect 4 output variables.
This output is identical to the output if each band is requested e.g. adding `variables=['Band1','Band2', 'Band3', 'Band4']` to the Request.


In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1849288272-ASF',
                Environment.UAT: 'G1244144595-EEDTEST',
                Environment.SIT: 'G1244144595-EEDTEST'}

    avnir_directory_five = 'avnir_test_five'
    
    avnir_request_five = Request(collection=environment_information['avnir_collection'],
                                 spatial=BBox(-99.2, 36.95, -98.9, 37.05), format='image/tiff',
                                 granule_id=granules[environment_information['env']])

    avnir_output_five = make_request_and_download_result(harmony_client, avnir_request_five,
                                                         avnir_directory_five)

    avnir_variable_base_five = 'ALAV2A278852850-OORIRFU'

    avnir_variables_five = {f'IMG-03-{avnir_variable_base_five}_000_band_1',
                            f'IMG-02-{avnir_variable_base_five}_000_band_1',
                            f'IMG-04-{avnir_variable_base_five}_000_band_1',
                            f'IMG-01-{avnir_variable_base_five}_000_band_1'}

    expected_avnir_results_five = {'authority': 'EPSG',
                                   'cs': 'Projected',
                                   'gcs': 'WGS 84',
                                   'gcs_epsg': '4326',
                                   'n_bands': 4,
                                   'proj_cs':'WGS 84 / UTM zone 14N',
                                   'proj_epsg': '32614',
                                   'spatial_extent': [36.9, 37.1, -99.18, -98.92],
                                   'variables': avnir_variables_five,
                                   'width': 2853,
                                   'height': 1693}
    
    check_request_output(avnir_output_five[0], expected_avnir_results_five)
    clean_test_artefacts(avnir_directory_five)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')   

### AVNIR test case 6:

This test combines a bounding box spatial subset in the Southern Hemisphere with a variable subset requesting bands 1 and 3 and specifies the height and width of the output bands.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving 2 output bands.


In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1835549401-ASF',
                Environment.UAT: 'G1244144601-EEDTEST',
                Environment.SIT: 'G1244144601-EEDTEST'}

    avnir_directory_six = 'avnir_test_six'
    
    avnir_request_six = Request(collection=environment_information['avnir_collection'],
                                spatial=BBox(18.07, -26.1, 18.25, -25.8),
                                variables=['Band1', 'Band3'], format='image/tiff',
                                granule_id=granules[environment_information['env']],
                                height=30, width=75)

    avnir_output_six = make_request_and_download_result(harmony_client, avnir_request_six,
                                                        avnir_directory_six)

    avnir_variable_base_six = 'ALAV2A151894120-OORIRFU'

    avnir_variables_six = {f'IMG-01-{avnir_variable_base_six}_000_band_1',
                           f'IMG-03-{avnir_variable_base_six}_000_band_1'}

    expected_avnir_results_six = {'authority': 'EPSG',
                                  'cs': 'Projected',
                                  'gcs': 'WGS 84',
                                  'gcs_epsg': '4326',
                                  'n_bands': 2,
                                  'proj_cs':'WGS 84 / UTM zone 34S',
                                  'proj_epsg': '32734',
                                  'spatial_extent': [-26.13, -25.77, 18.07, 18.25],
                                  'variables': avnir_variables_six,
                                  'width': 75,
                                  'height': 30}
    
    check_request_output(avnir_output_six[0], expected_avnir_results_six)
    clean_test_artefacts(avnir_directory_six)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 7:

This test combines a bounding box spatial subset (with negative longitude values) with all bands being explicitly specified alongside the height and width of the output array.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving all 4 output bands.


In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1906378530-ASF',
                Environment.UAT: 'G1244144603-EEDTEST',
                Environment.SIT: 'G1244144603-EEDTEST'}

    avnir_directory_seven = 'avnir_test_seven'
    
    avnir_request_seven = Request(collection=environment_information['avnir_collection'],
                                  spatial=BBox(-97.65, 49.65, -97.35, 49.95),
                                  variables=['Band1', 'Band2', 'Band3', 'Band4'],
                                  granule_id=granules[environment_information['env']],
                                  height=45, width=90, format='image/tiff')

    avnir_output_seven = make_request_and_download_result(harmony_client, avnir_request_seven,
                                                          avnir_directory_seven)

    avnir_variable_base_seven = 'ALAV2A277832590-OORIRFU'

    avnir_variables_seven = {f'IMG-01-{avnir_variable_base_seven}_000_band_1',
                             f'IMG-02-{avnir_variable_base_seven}_000_band_1',
                             f'IMG-04-{avnir_variable_base_seven}_000_band_1',
                             f'IMG-03-{avnir_variable_base_seven}_000_band_1',}

    expected_avnir_results_seven = {'authority': 'EPSG',
                                    'cs': 'Projected',
                                    'gcs': 'WGS 84',
                                    'gcs_epsg': '4326',
                                    'n_bands': 4,
                                    'proj_cs':'WGS 84 / UTM zone 14N',
                                    'proj_epsg': '32614',
                                    'spatial_extent': [49.6, 49.99, -97.64, -97.36],
                                    'variables': avnir_variables_seven,
                                    'width': 90,
                                    'height': 45}
    
    check_request_output(avnir_output_seven[0], expected_avnir_results_seven)
    clean_test_artefacts(avnir_directory_seven)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 8:

This test requests a bounding box spatial subset, where the bounding box is larger than granule itself. No variables are specified, meaning the request should return all variables.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are retrieving all 4 output bands.

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1813329318-ASF',
                Environment.UAT: 'G1244144596-EEDTEST',
                Environment.SIT: 'G1244144596-EEDTEST'}

    avnir_directory_eight = 'avnir_test_eight'
    
    avnir_request_eight = Request(collection=environment_information['avnir_collection'],
                                  spatial=BBox(-79.9, 9.2, -78.9, 10.3),
                                  granule_id=granules[environment_information['env']],
                                  format='image/tiff')

    avnir_output_eight = make_request_and_download_result(harmony_client, avnir_request_eight,
                                                          avnir_directory_eight)

    avnir_variable_base_eight = 'ALAV2A150623400-OORIRFU'

    avnir_variables_eight = { f'IMG-01-{avnir_variable_base_eight}_000_band_1',
                              f'IMG-02-{avnir_variable_base_eight}_000_band_1',
                              f'IMG-04-{avnir_variable_base_eight}_000_band_1',
                              f'IMG-03-{avnir_variable_base_eight}_000_band_1', }

    expected_avnir_results_eight = {'authority': 'EPSG',
                                    'cs': 'Projected',
                                    'gcs': 'WGS 84',
                                    'gcs_epsg': '4326',
                                    'n_bands': 4,
                                    'proj_cs':'WGS 84 / UTM zone 17N',
                                    'proj_epsg': '32617',
                                    'spatial_extent': [9.4, 10.24, -79.7, -79.2],
                                    'variables': avnir_variables_eight,
                                    'width': 7215,
                                    'height': 8000}
    
    check_request_output(avnir_output_eight[0], expected_avnir_results_eight)
    clean_test_artefacts(avnir_directory_eight)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 9:

This test combines a bounding box spatial subset with a temporal range subset. All variables are explicitly specified. No granules are specified in this request, meaning the results will be determined by those granules that match the spatial and temporal criteria.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. In this test, we are requesting "all" variables and expecting to retrieve all 4 output bands.


In [ ]:
if environment_information is not None:
    avnir_directory_nine = 'avnir_test_nine'
    
    avnir_request_nine = Request(collection=environment_information['avnir_collection'],
                                 variables=['all'],
                                 spatial=BBox(18.4, -26.1, 18.6, -26.0),
                                 format='image/tiff')

    avnir_output_nine = make_request_and_download_result(harmony_client, avnir_request_nine,
                                                         avnir_directory_nine)

    avnir_variable_base_nine = 'ALAV2A151894120-OORIRFU'

    avnir_variables_nine = {f'IMG-01-{avnir_variable_base_nine}_000_band_1',
                            f'IMG-02-{avnir_variable_base_nine}_000_band_1',
                            f'IMG-03-{avnir_variable_base_nine}_000_band_1',
                            f'IMG-04-{avnir_variable_base_nine}_000_band_1'}

    expected_avnir_results_nine = {'authority': 'EPSG',
                                   'cs': 'Projected',
                                   'gcs': 'WGS 84',
                                   'gcs_epsg': '4326',
                                   'n_bands': 4,
                                   'proj_cs':'WGS 84 / UTM zone 34S',
                                   'proj_epsg': '32734',
                                   'spatial_extent': [-26.13, -25.96, 18.41, 18.59],
                                   'variables': avnir_variables_nine,
                                   'width': 2195,
                                   'height': 1516}

    check_request_output(avnir_output_nine[0], expected_avnir_results_nine)
    clean_test_artefacts(avnir_directory_nine)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 10:

This test combines a bounding box spatial subset, across the Equator, with a variable subset for a single variable. The output should be in a `.png` format.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. 

For a `.png` output, the authority and projection can't be derived from the output file, so several of the tested properties are `None` for this test case.

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1813212660-ASF',
                Environment.UAT: 'G1244144597-EEDTEST',
                Environment.SIT: 'G1244144597-EEDTEST'}

    avnir_directory_ten = 'avnir_test_ten'
    
    avnir_request_ten = Request(collection=environment_information['avnir_collection'],
                                spatial=BBox(-51.0, -0.05, -50.75, 0.25), variables=['Band2'],
                                granule_id=granules[environment_information['env']],
                                format='image/png')

    avnir_output_ten = make_request_and_download_result(harmony_client, avnir_request_ten,
                                                        avnir_directory_ten)

    avnir_variables_ten = {'Band1', 'Band2', 'Band3', 'Band4'}

    expected_avnir_results_ten = {'authority': None,
                                  'cs': None,
                                  'file_type': 'png',
                                  'gcs': None,
                                  'gcs_epsg': None,
                                  'n_bands': 4,
                                  'proj_cs': None,
                                  'proj_epsg': None,
                                  'spatial_extent': None,
                                  'variables': avnir_variables_ten,
                                  'width': 3410,
                                  'height': 3238}
    
    check_request_output(avnir_output_ten[0], expected_avnir_results_ten)
    clean_test_artefacts(avnir_directory_ten)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### AVNIR test case 11:
This test combines a bounding box spatial subset, across the Equator, with a variable subset for a single variable. The output should be in a `.gif` format.

The input `.zip` file contains 4 Image GeoTIFFs, one per band each representing a spectrum range. 


In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1813212660-ASF',
                Environment.UAT: 'G1244144597-EEDTEST',
                Environment.SIT: 'G1244144597-EEDTEST'}
    avnir_directory_eleven = 'avnir_test_eleven'
    
    avnir_request_eleven = Request(collection=environment_information['avnir_collection'],
                                   spatial=BBox(-51.0, -0.05, -50.75, 0.25), variables=['Band2'],
                                   granule_id=granules[environment_information['env']],
                                   format='image/gif')

    avnir_output_eleven = make_request_and_download_result(harmony_client, avnir_request_eleven,
                                                           avnir_directory_eleven)
    expected_avnir_results_eleven = {'authority': None,
                                     'cs': None,
                                     'file_type': 'gif',
                                     'gcs': None,
                                     'gcs_epsg': None,
                                     'n_bands': 1,
                                     'proj_cs': None,
                                     'proj_epsg': None,
                                     'spatial_extent': None,
                                     'variables': {},
                                     'reference_image': 'reference_images/avnir_reference_eleven.gif',
                                     'width': 3410,
                                     'height': 3238}

    check_request_output(avnir_output_eleven[0], expected_avnir_results_eleven)
    clean_test_artefacts(avnir_directory_eleven)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

## Sentinel 1 Interferograms:

The tests below will use the Sentinel 1 Interferograms collection (for UAT this is mirrored in the EEDTEST CMR environment).

### Sentinel test case 1:

This test combines a bounding box spatial subset with a variable subset (`/science/grids/data/amplitude`). The input NetCDF-4 file contains many variables (in a hierarchical structure).

The output NetCDF-4 file is expected to have four colour bands per requested variable, alongside coordinate variables. This leads to 7 total output variables (4 colour bands, latitude, longitude and CRS).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1715962900-ASF',
                Environment.UAT: 'G1244144618-EEDTEST',
                Environment.SIT: 'G1244144618-EEDTEST'}

    sentinel_directory_one = 'sentinel_test_one'
    
    sentinel_request_one = Request(collection=environment_information['sentinel_collection'],
                                   spatial=BBox(-115.5, 33, -115.25, 33.1),
                                   variables=['science/grids/data/amplitude'],
                                   granule_id=granules[environment_information['env']])

    sentinel_output_one = make_request_and_download_result(harmony_client, sentinel_request_one,
                                                           sentinel_directory_one)

    sentinel_base_variable_one = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2_'
                                  'science_grids_data_amplitude_subsetted__'
                                  'science_grids_data_amplitude__colored_band')

    sentinel_variables_one = {f'{sentinel_base_variable_one}_1',
                              f'{sentinel_base_variable_one}_2',
                              f'{sentinel_base_variable_one}_3',
                              f'{sentinel_base_variable_one}_4',
                              'lat',
                              'lon',
                              'latitude_longitude'}

    expected_sentinel_results_one = {'authority': None,
                                     'cs': None,
                                     'gcs': None,
                                     'gcs_epsg': None,
                                     'n_bands': 7,
                                     'proj_cs': None,
                                     'proj_epsg': None,
                                     'reference_image': 'reference_images/sentinel_reference_one.nc',
                                     'spatial_extent': [33.0, 33.1, -115.5, -115.25],
                                     'variables': sentinel_variables_one,
                                     'width': 300,
                                     'height': 120}
    
    check_request_output(sentinel_output_one[0], expected_sentinel_results_one)
    clean_test_artefacts(sentinel_directory_one)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### Sentinel test case 2:

This test combines a bounding box spatial subset with a variable subset (`/science/grids/data/coherence`). The output format will be GeoTIFF.

The input NetCDF-4 file contains many variables (in a hierarchical structure). The output GeoTIFF is expected to have four colour bands per requested variable, this leads to 4 total output bands.

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1944514838-ASF',
                Environment.UAT: 'G1244144611-EEDTEST',
                Environment.SIT: 'G1244144611-EEDTEST'}

    sentinel_directory_two = 'sentinel_test_two'
    
    sentinel_request_two = Request(collection=environment_information['sentinel_collection'],
                                   spatial=BBox(37.25, 26.95, 38.15, 27.25),
                                   variables=['science/grids/data/coherence'],
                                   granule_id=granules[environment_information['env']],
                                   format='image/tiff')

    sentinel_output_two = make_request_and_download_result(harmony_client, sentinel_request_two,
                                                           sentinel_directory_two)

    sentinel_base_variable_two = ('S1-GUNW-A-R-014-tops-20191127_20191115-153047-28315N_26592N-PP-8a6f-v2_0_3_'
                                  'science_grids_data_coherence_subsetted__'
                                  'science_grids_data_coherence__colored_band')

    sentinel_variables_two = {f'{sentinel_base_variable_two}_1',
                              f'{sentinel_base_variable_two}_2',
                              f'{sentinel_base_variable_two}_3',
                              f'{sentinel_base_variable_two}_4'}

    expected_sentinel_results_two = {'authority': 'EPSG',
                                     'cs': 'Geographic',
                                     'gcs': 'WGS 84',
                                     'gcs_epsg': '4326',
                                     'n_bands': 4,
                                     'proj_cs': None,
                                     'proj_epsg': None,
                                     'reference_image': 'reference_images/sentinel_reference_two.tiff',
                                     'spatial_extent': [26.95, 27.25, 37.25, 38.15],
                                     'variables': sentinel_variables_two,
                                     'width': 1080,
                                     'height': 360}
    
    check_request_output(sentinel_output_two[0], expected_sentinel_results_two)
    clean_test_artefacts(sentinel_directory_two)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### Sentinel test case 3:

This test combines a bounding box spatial subset with a variable subset (`/science/grids/data/connectedComponents`). 

The input NetCDF-4 file contains many variables (in a hierarchical structure). The output NetCDF-4 is expected to have four colour bands per requested variable, alongside coordinate variables. This leads to 7 total output variables (4 colour bands, latitude, longitude and CRS).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1944514838-ASF',
                Environment.UAT: 'G1244144611-EEDTEST',
                Environment.SIT: 'G1244144611-EEDTEST'}

    sentinel_directory_three = 'sentinel_test_three'
    
    sentinel_request_three = Request(collection=environment_information['sentinel_collection'],
                                     spatial=BBox(37.5, 27.0, 38.25, 27.5),
                                     variables=['science/grids/data/connectedComponents'],
                                     granule_id=granules[environment_information['env']])

    sentinel_output_three = make_request_and_download_result(harmony_client, sentinel_request_three,
                                                             sentinel_directory_three)

    sentinel_base_variable_three = ('S1_GUNW_A_R_014_tops_20191127_20191115_153047_28315N_26592N_PP_8a6f_v2_0_3_'
                                    'science_grids_data_connectedComponents_subsetted__'
                                    'science_grids_data_connectedComponents__colored_band')

    sentinel_variables_three = {f'{sentinel_base_variable_three}_1',
                                f'{sentinel_base_variable_three}_2',
                                f'{sentinel_base_variable_three}_3',
                                f'{sentinel_base_variable_three}_4',
                                'lat',
                                'lon',
                                'latitude_longitude'}

    expected_sentinel_results_three = {'authority': None,
                                       'cs': None,
                                       'gcs': None,
                                       'gcs_epsg': None,
                                       'n_bands': 7,
                                       'proj_cs': None,
                                       'proj_epsg': None,
                                       'reference_image': 'reference_images/sentinel_reference_three.nc',
                                       'spatial_extent': [27.0, 27.5, 37.5, 38.25],
                                       'variables': sentinel_variables_three,
                                       'width': 900,
                                       'height': 600}
    
    check_request_output(sentinel_output_three[0], expected_sentinel_results_three)
    clean_test_artefacts(sentinel_directory_three)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### Sentinel test case 4:

This test combines a bounding box spatial subset with a variable subset (`/science/grids/data/unwrappedPhase`). The input NetCDF-4 file contains many variables (in a hierarchical structure).

The output NetCDF-4 is expected to have four colour bands per requested variable, alongside coordinate variables. This leads to 7 total output variables (4 colour bands, latitude, longitude and CRS).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1944514838-ASF',
                Environment.UAT: 'G1244144611-EEDTEST',
                Environment.SIT: 'G1244144611-EEDTEST'}

    sentinel_directory_four = 'sentinel_test_four'
    
    sentinel_request_four = Request(collection=environment_information['sentinel_collection'],
                                    spatial=BBox(36.9, 26.9, 39.1, 27.5),
                                    variables=['science/grids/data/unwrappedPhase'],
                                    granule_id=granules[environment_information['env']])

    sentinel_output_four = make_request_and_download_result(harmony_client, sentinel_request_four,
                                                            sentinel_directory_four)

    sentinel_base_variable_four = ('S1_GUNW_A_R_014_tops_20191127_20191115_153047_28315N_26592N_PP_8a6f_v2_0_3_'
                                   'science_grids_data_unwrappedPhase_subsetted__'
                                   'science_grids_data_unwrappedPhase__colored_band')

    sentinel_variables_four = {f'{sentinel_base_variable_four}_1',
                               f'{sentinel_base_variable_four}_2',
                               f'{sentinel_base_variable_four}_3',
                               f'{sentinel_base_variable_four}_4',
                               'lat',
                               'lon',
                               'latitude_longitude'}

    expected_sentinel_results_four = {'authority': None,
                                      'cs': None,
                                      'gcs': None,
                                      'gcs_epsg': None,
                                      'n_bands': 7,
                                      'proj_cs': None,
                                      'proj_epsg': None,
                                      'reference_image': 'reference_images/sentinel_reference_four.nc',
                                      'spatial_extent': [26.9, 27.5, 36.9, 39.1],
                                      'variables': sentinel_variables_four,
                                      'width': 2640,
                                      'height': 720}
    
    check_request_output(sentinel_output_four[0], expected_sentinel_results_four)
    clean_test_artefacts(sentinel_directory_four)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### Sentinel test case 5:

This test combines a bounding box spatial subset with a variable subset (`/science/grids/data/amplitude` and `/science/grids/data/coherence`). Additionally, the height and width of the output variables are specified.

The input NetCDF-4 file contains many variables (in a hierarchical structure).

The output NetCDF-4 is expected to have four colour bands per requested variable, alongside coordinate variables. This leads to 11 total output variables (4 colour bands for each of the `amplitude` and `coherence`, latitude, longitude and CRS).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1715962900-ASF',
                Environment.UAT: 'G1244144618-EEDTEST',
                Environment.SIT: 'G1244144618-EEDTEST'}

    sentinel_directory_five = 'sentinel_test_five'

    sentinel_request_five = Request(collection=environment_information['sentinel_collection'],
                                    spatial=BBox(-115.5, 33, -115.25, 33.1),
                                    variables=['science/grids/data/amplitude',
                                               'science/grids/data/coherence'],
                                    granule_id=granules[environment_information['env']],
                                    height=60, width=150)

    sentinel_output_five = make_request_and_download_result(harmony_client, sentinel_request_five,
                                                            sentinel_directory_five)

    sentinel_base_amplitude_five = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2_'
                                    'regridded_subsetted__science_grids_data_amplitude__colored_band')
    sentinel_base_coherence_five = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2'
                                    '_regridded_subsetted__science_grids_data_coherence__colored_band')

    sentinel_variables_five = {f'{sentinel_base_amplitude_five}_1',
                               f'{sentinel_base_amplitude_five}_2',
                               f'{sentinel_base_amplitude_five}_3',
                               f'{sentinel_base_amplitude_five}_4',
                               f'{sentinel_base_coherence_five}_1',
                               f'{sentinel_base_coherence_five}_2',
                               f'{sentinel_base_coherence_five}_3',
                               f'{sentinel_base_coherence_five}_4',
                               'lat',
                               'lon',
                               'latitude_longitude'}

    expected_sentinel_results_five = {'authority': None,
                                      'cs': None,
                                      'gcs': None,
                                      'gcs_epsg': None,
                                      'n_bands': 11,
                                      'proj_cs': None,
                                      'proj_epsg': None,
                                      'reference_image': 'reference_images/sentinel_reference_five.nc',
                                      'spatial_extent': [33.0, 33.1, -115.5, -115.25],
                                      'variables': sentinel_variables_five,
                                      'width': 150,
                                      'height': 60}
    
    check_request_output(sentinel_output_five[0], expected_sentinel_results_five)
    clean_test_artefacts(sentinel_directory_five)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### Sentinel test case 6:

This test combines a bounding box spatial subset with a variable subset (`/science/grids/data/amplitude`, `/science/grids/data/coherence` and `/science/grids/data/connectedComponents`). Additionally, the height and width of the output variables are specified.

The input NetCDF-4 file contains many variables (in a hierarchical structure). The output NetCDF-4 is expected to have four colour bands per requested variable, alongside coordinate variables. This leads to 15 total output variables (4 colour bands for each of the `amplitude`, `coherence` and `connectedComponents`, latitude, longitude and CRS).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1715962900-ASF',
                Environment.UAT: 'G1244144618-EEDTEST',
                Environment.SIT: 'G1244144618-EEDTEST'}

    sentinel_directory_six = 'sentinel_test_six'

    sentinel_request_six = Request(collection=environment_information['sentinel_collection'],
                                   spatial=BBox(-115.52, 33.05, -115.27, 33.1),
                                   variables=['science/grids/data/amplitude',
                                              'science/grids/data/coherence',
                                              'science/grids/data/connectedComponents'],
                                   granule_id=granules[environment_information['env']],
                                   height=30, width=75)

    sentinel_output_six = make_request_and_download_result(harmony_client, sentinel_request_six,
                                                           sentinel_directory_six)

    sentinel_base_amplitude_six = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2_'
                                    'regridded_subsetted__science_grids_data_amplitude__colored_band')
    sentinel_base_coherence_six = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2'
                                    '_regridded_subsetted__science_grids_data_coherence__colored_band')
    sentinel_base_components_six = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2'
                                    '_regridded_subsetted__science_grids_data_connectedComponents__colored_band')

    sentinel_variables_six = {f'{sentinel_base_amplitude_six}_1',
                              f'{sentinel_base_amplitude_six}_2',
                              f'{sentinel_base_amplitude_six}_3',
                              f'{sentinel_base_amplitude_six}_4',
                              f'{sentinel_base_coherence_six}_1',
                              f'{sentinel_base_coherence_six}_2',
                              f'{sentinel_base_coherence_six}_3',
                              f'{sentinel_base_coherence_six}_4',
                              f'{sentinel_base_components_six}_1',
                              f'{sentinel_base_components_six}_2',
                              f'{sentinel_base_components_six}_3',
                              f'{sentinel_base_components_six}_4',
                              'lat',
                              'lon',
                              'latitude_longitude'}

    expected_sentinel_results_six = {'authority': None,
                                     'cs': None,
                                     'gcs': None,
                                     'gcs_epsg': None,
                                     'n_bands': 15,
                                     'proj_cs': None,
                                     'proj_epsg': None,
                                     'reference_image': 'reference_images/sentinel_reference_six.nc',
                                     'spatial_extent': [33.05, 33.1, -115.52, -115.27],
                                     'variables': sentinel_variables_six,
                                     'width': 75,
                                     'height': 30}
    
    check_request_output(sentinel_output_six[0], expected_sentinel_results_six)
    clean_test_artefacts(sentinel_directory_six)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### Sentinel test case 7:

This test combines a bounding box spatial subset with a variable subset (`/science/grids/data/amplitude`, `/science/grids/data/coherence`, `/science/grids/data/connectedComponents` and `/science/grids/data/unwrappedPhase`).

The input NetCDF-4 file contains many variables (in a hierarchical structure). The output is expected to have four colour bands per requested variable, alongside coordinate variables. This leads to 19 total output variables (4 colour bands for each of `amplitude`, `coherence`, `connectedComponents` and `unwrappedPhase`, latitude, longitude and CRS).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1715962900-ASF',
                Environment.UAT: 'G1244144618-EEDTEST',
                Environment.SIT: 'G1244144618-EEDTEST'}

    sentinel_directory_seven = 'sentinel_test_seven'

    sentinel_request_seven = Request(collection=environment_information['sentinel_collection'],
                                     spatial=BBox(-115.45, 32.95, -115.15, 33.09),
                                     variables=['science/grids/data/amplitude',
                                                'science/grids/data/coherence',
                                                'science/grids/data/connectedComponents',
                                                'science/grids/data/unwrappedPhase'],
                                     granule_id=granules[environment_information['env']])

    sentinel_output_seven = make_request_and_download_result(harmony_client, sentinel_request_seven,
                                                             sentinel_directory_seven)

    sentinel_base_amplitude_seven = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2'
                                     '_subsetted__science_grids_data_amplitude__colored_band')
    sentinel_base_coherence_seven = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2'
                                     '_subsetted__science_grids_data_coherence__colored_band')
    sentinel_base_components_seven = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2'
                                      '_subsetted__science_grids_data_connectedComponents__colored_band')
    sentinel_base_phase_seven = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2'
                                 '_subsetted__science_grids_data_unwrappedPhase__colored_band')

    sentinel_variables_seven = {f'{sentinel_base_amplitude_seven}_1',
                                f'{sentinel_base_amplitude_seven}_2',
                                f'{sentinel_base_amplitude_seven}_3',
                                f'{sentinel_base_amplitude_seven}_4',
                                f'{sentinel_base_coherence_seven}_1',
                                f'{sentinel_base_coherence_seven}_2',
                                f'{sentinel_base_coherence_seven}_3',
                                f'{sentinel_base_coherence_seven}_4',
                                f'{sentinel_base_components_seven}_1',
                                f'{sentinel_base_components_seven}_2',
                                f'{sentinel_base_components_seven}_3',
                                f'{sentinel_base_components_seven}_4',
                                f'{sentinel_base_phase_seven}_1',
                                f'{sentinel_base_phase_seven}_2',
                                f'{sentinel_base_phase_seven}_3',
                                f'{sentinel_base_phase_seven}_4',
                                'lat',
                                'lon',
                                'latitude_longitude'}

    expected_sentinel_results_seven = {'authority': None,
                                       'cs': None,
                                       'gcs': None,
                                       'gcs_epsg': None,
                                       'n_bands': 19,
                                       'proj_cs': None,
                                       'proj_epsg': None,
                                       'reference_image': 'reference_images/sentinel_reference_seven.nc',
                                       'spatial_extent': [32.95, 33.09, -115.45, -115.15],
                                       'variables': sentinel_variables_seven,
                                       'width': 360,
                                       'height': 168}
    
    check_request_output(sentinel_output_seven[0], expected_sentinel_results_seven)
    clean_test_artefacts(sentinel_directory_seven)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### Sentinel test case 8:

This test combines a bounding box spatial subset in the Southern Hemisphere (negative latitudes and longitudes) with a variable subset (`/science/grids/data/amplitude`). The bounding box is smaller than the extent of the input granule.

The input NetCDF-4 file contains many variables (in a hierarchical structure). The output is expected to have four colour bands per requested variable, alongside coordinate variables. This leads to 7 total output variables (4 colour bands for `amplitude`, latitude, longitude and CRS).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1697222082-ASF',
                Environment.UAT: 'G1244144612-EEDTEST',
                Environment.SIT: 'G1244144612-EEDTEST'}

    sentinel_directory_eight = 'sentinel_test_eight'

    sentinel_request_eight = Request(collection=environment_information['sentinel_collection'],
                                     spatial=BBox(-69.8, -37.2, -68.8, -36.9),
                                     variables=['science/grids/data/amplitude'],
                                     granule_id=granules[environment_information['env']])

    sentinel_output_eight = make_request_and_download_result(harmony_client, sentinel_request_eight,
                                                             sentinel_directory_eight)

    sentinel_base_variable_eight = ('S1_GUNW_A_R_018_tops_20200201_20190131_232716_36003S_38134S_PP_92d7_v2_0_2'
                                    '_science_grids_data_amplitude_subsetted__'
                                    'science_grids_data_amplitude__colored_band')

    sentinel_variables_eight = {f'{sentinel_base_variable_eight}_1',
                                f'{sentinel_base_variable_eight}_2',
                                f'{sentinel_base_variable_eight}_3',
                                f'{sentinel_base_variable_eight}_4',
                                'lat',
                                'lon',
                                'latitude_longitude'}

    expected_sentinel_results_eight = {'authority': None,
                                       'cs': None,
                                       'gcs': None,
                                       'gcs_epsg': None,
                                       'n_bands': 7,
                                       'proj_cs': None,
                                       'proj_epsg': None,
                                       'reference_image': 'reference_images/sentinel_reference_eight.nc',
                                       'spatial_extent': [-37.2, -36.9, -69.8, -68.8],
                                       'variables': sentinel_variables_eight,
                                       'width': 1200,
                                       'height': 360}
    
    check_request_output(sentinel_output_eight[0], expected_sentinel_results_eight)
    clean_test_artefacts(sentinel_directory_eight)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### Sentinel test case 9:

This test combines a bounding box spatial subset in the Southern Hemisphere (negative latitudes and longitudes) with a variable subset (`/science/grids/data/coherence`). The bounding box is larger than the extent of the input granule.

The input NetCDF-4 file contains many variables (in a hierarchical structure). The output is expected to have four colour bands per requested variable, alongside coordinate variables. This leads to 7 total output variables (4 colour bands for `coherence`, latitude, longitude and CRS).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1697222082-ASF',
                Environment.UAT: 'G1244144612-EEDTEST',
                Environment.SIT: 'G1244144612-EEDTEST'}

    sentinel_directory_nine = 'sentinel_test_nine'

    sentinel_request_nine = Request(collection=environment_information['sentinel_collection'],
                                    spatial=BBox(-71.6, -39.0, -67.5, -35.0),
                                    variables=['science/grids/data/coherence'],
                                    granule_id=granules[environment_information['env']])

    sentinel_output_nine = make_request_and_download_result(harmony_client, sentinel_request_nine,
                                                            sentinel_directory_nine)

    sentinel_base_variable_nine = ('S1_GUNW_A_R_018_tops_20200201_20190131_232716_36003S_38134S_PP_92d7_v2_0_2'
                                   '_science_grids_data_coherence_subsetted__'
                                   'science_grids_data_coherence__colored_band')

    sentinel_variables_nine = {f'{sentinel_base_variable_nine}_1',
                               f'{sentinel_base_variable_nine}_2',
                               f'{sentinel_base_variable_nine}_3',
                               f'{sentinel_base_variable_nine}_4',
                               'lat',
                               'lon',
                               'latitude_longitude'}

    expected_sentinel_results_nine = {'authority': None,
                                      'cs': None,
                                      'gcs': None,
                                      'gcs_epsg': None,
                                      'n_bands': 7,
                                      'proj_cs': None,
                                      'proj_epsg': None,
                                      'reference_image': 'reference_images/sentinel_reference_nine.nc',
                                      'spatial_extent': [-38.14, -36.0, -71.41, -67.96],
                                      'variables': sentinel_variables_nine,
                                      'width': 4143,
                                      'height': 2559}
    
    check_request_output(sentinel_output_nine[0], expected_sentinel_results_nine)
    clean_test_artefacts(sentinel_directory_nine)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### Sentinel test case 10:

This test combines a bounding box spatial subset of China with a variable subset (`/science/grids/data/coherence` and `/science/grids/data/unwrappedPhase`). The bounding box is smaller than the extent of the input granule.

The input NetCDF-4 file contains many variables (in a hierarchical structure). The output is expected to have four colour bands per requested variable, alongside coordinate variables. This leads to 11 total output variables (4 colour bands for each of `coherence` and `unwrappedPhase`, latitude, longitude and CRS).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1722540208-ASF',
                Environment.UAT: 'G1244144616-EEDTEST',
                Environment.SIT: 'G1244144616-EEDTEST'}

    sentinel_directory_ten = 'sentinel_test_ten'

    sentinel_request_ten = Request(collection=environment_information['sentinel_collection'],
                                   spatial=BBox(86.85, 43.75, 88.15, 44.85),
                                   variables=['science/grids/data/coherence',
                                              'science/grids/data/unwrappedPhase'],
                                     granule_id=granules[environment_information['env']])

    sentinel_output_ten = make_request_and_download_result(harmony_client, sentinel_request_ten,
                                                           sentinel_directory_ten)

    sentinel_base_coherence_ten = ('S1_GUNW_A_R_041_tops_20191123_20191030_120209_45050N_43180N_PP_0ea8_v2_0_2'
                                   '_subsetted__science_grids_data_coherence__colored_band')
    sentinel_base_phase_ten = ('S1_GUNW_A_R_041_tops_20191123_20191030_120209_45050N_43180N_PP_0ea8_v2_0_2'
                               '_subsetted__science_grids_data_unwrappedPhase__colored_band')

    sentinel_variables_ten = {f'{sentinel_base_coherence_ten}_1',
                              f'{sentinel_base_coherence_ten}_2',
                              f'{sentinel_base_coherence_ten}_3',
                              f'{sentinel_base_coherence_ten}_4',
                              f'{sentinel_base_phase_ten}_1',
                              f'{sentinel_base_phase_ten}_2',
                              f'{sentinel_base_phase_ten}_3',
                              f'{sentinel_base_phase_ten}_4',
                              'lat',
                              'lon',
                              'latitude_longitude'}

    expected_sentinel_results_ten = {'authority': None,
                                     'cs': None,
                                     'gcs': None,
                                     'gcs_epsg': None,
                                     'n_bands': 11,
                                     'proj_cs': None,
                                     'proj_epsg': None,
                                     'reference_image': 'reference_images/sentinel_reference_ten.nc',
                                     'spatial_extent': [43.75, 44.85, 86.85, 88.15],
                                     'variables': sentinel_variables_ten,
                                     'width': 1560,
                                     'height': 1320}
    
    check_request_output(sentinel_output_ten[0], expected_sentinel_results_ten)
    clean_test_artefacts(sentinel_directory_ten)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### Sentinel test case 11:

This test combines a bounding box spatial subset with a variable subset (`/science/grids/data/amplitude`). The output file format will be a `.png`.

The input NetCDF-4 file contains many variables (in a hierarchical structure). The output is expected to have four colour bands per requested variable, alongside coordinate variables. This leads to 4 total output variables (4 colour bands for `amplitude`).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1715962900-ASF',
                Environment.UAT: 'G1244144618-EEDTEST',
                Environment.SIT: 'G1244144618-EEDTEST'}

    sentinel_directory_eleven = 'sentinel_test_eleven'

    sentinel_request_eleven = Request(collection=environment_information['sentinel_collection'],
                                      spatial=BBox(-115.5, 33, -115.45, 33.05),
                                      variables=['science/grids/data/amplitude'],
                                      granule_id=granules[environment_information['env']],
                                      format='image/png')

    sentinel_output_eleven = make_request_and_download_result(harmony_client, sentinel_request_eleven,
                                                              sentinel_directory_eleven)

    sentinel_variables_eleven = {'Band1', 'Band2', 'Band3', 'Band4'}

    expected_sentinel_results_eleven = {'authority': None,
                                        'cs': None,
                                        'file_type': 'png',
                                        'gcs': None,
                                        'gcs_epsg': None,
                                        'n_bands': 4,
                                        'proj_cs': None,
                                        'proj_epsg': None,
                                        'reference_image': 'reference_images/sentinel_reference_eleven.png',
                                        'spatial_extent': None,
                                        'variables': sentinel_variables_eleven,
                                        'width': 60,
                                        'height': 60}
    
    check_request_output(sentinel_output_eleven[0], expected_sentinel_results_eleven)
    clean_test_artefacts(sentinel_directory_eleven)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

## Sentinel test case 12:

This test combines a bounding box spatial subset with a variable subset (`/science/grids/data/amplitude`). The requested output is a `.gif`.

# TO FIX: Request currently gets `gdal_translate` error:

```
2022-04-07T23:15:57.153Z [debug]: INFO:root:gdal_translate '-of' 'GIF' '-scale' '/tmp/tmpy79d8sb1/result.tif' '/tmp/tmpy79d8sb1/translated.gif'
2022-04-07T23:15:57.236Z [debug]: ERROR 6: GIF driver only supports one band images.
```

In [ ]:
"""
if environment_information is not None:
    granules = {Environment.PROD: 'G1715962900-ASF',
                Environment.UAT: 'G1244144618-EEDTEST',
                Environment.SIT: 'G1244144618-EEDTEST'}

    sentinel_directory_twelve = 'sentinel_test_twelve'

    sentinel_request_twelve = Request(collection=environment_information['sentinel_collection'],
                                      spatial=BBox(-115.5, 33, -115.45, 33.05),
                                      variables=['science/grids/data/amplitude'],
                                      granule_id=granules[environment_information['env']],
                                      format='image/gif')

    sentinel_output_twelve = make_request_and_download_result(harmony_client, sentinel_request_twelve,
                                                              sentinel_directory_twelve)

    sentinel_variables_twelve = {}

    expected_sentinel_results_twelve = {'authority': None,
                                        'cs': None,
                                        'file_type': 'gif',
                                        'gcs': None,
                                        'gcs_epsg': None,
                                        'n_bands': 0,
                                        'proj_cs': None,
                                        'proj_epsg': None,
                                        # 'reference_image': 'reference_images/sentinel_reference_twelve.gif',
                                        'spatial_extent': None,
                                        'variables': sentinel_variables_twelve,
                                        'width': 60,
                                        'height': 60}
    
    check_request_output(sentinel_output_twelve[0], expected_sentinel_results_twelve)
    clean_test_artefacts(sentinel_directory_twelve)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')
"""

### Sentinel test case 13:

This test requests a bounding box spatial subset while not specifying any variable subset, so all variables should be returned. The output from this test should be identical to Sentinel test case 7.

The input NetCDF-4 file contains many variables (in a hierarchical structure). The output is expected to have four colour bands per requested variable, alongside coordinate variables. This leads to 19 total output variables (4 colour bands for each of `amplitude`, `coherence`, `connectedComponents` and `unwrappedPhase`, latitude, longitude and CRS).

# TO FIX: The request completes successfully, but doesn't create an entry in the output STAC.

**Also, there is a danger the same issue as GeoTIFF "all" requests exists - that only the first band is copied to the output.**

In [ ]:
"""
if environment_information is not None:
    granules = {Environment.PROD: 'G1715962900-ASF',
                Environment.UAT: 'G1244144618-EEDTEST',
                Environment.SIT: 'G1244144618-EEDTEST'}

    sentinel_directory_thirteen = 'sentinel_test_thirteen'

    sentinel_request_thirteen = Request(collection=environment_information['sentinel_collection'],
                                        spatial=BBox(-115.45, 32.95, -115.15, 33.09),
                                        granule_id=granules[environment_information['env']])

    sentinel_output_thirteen = make_request_and_download_result(harmony_client, sentinel_request_thirteen,
                                                                sentinel_directory_thirteen)

    sentinel_base_amplitude_thirteen = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2'
                                        '_subsetted__science_grids_data_amplitude__colored_band')
    sentinel_base_coherence_thirteen = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2'
                                        '_subsetted__science_grids_data_coherence__colored_band')
    sentinel_base_components_thirteen = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2'
                                        '_subsetted__science_grids_data_connectedComponents__colored_band')
    sentinel_base_phase_thirteen = ('S1_GUNW_A_R_166_tops_20200313_20200206_014119_34455N_32574N_PP_1749_v2_0_2'
                                    '_subsetted__science_grids_data_unwrappedPhase__colored_band')

    sentinel_variables_thirteen = {f'{sentinel_base_amplitude_thirteen}_1',
                                   f'{sentinel_base_amplitude_thirteen}_2',
                                   f'{sentinel_base_amplitude_thirteen}_3',
                                   f'{sentinel_base_amplitude_thirteen}_4',
                                   f'{sentinel_base_coherence_thirteen}_1',
                                   f'{sentinel_base_coherence_thirteen}_2',
                                   f'{sentinel_base_coherence_thirteen}_3',
                                   f'{sentinel_base_coherence_thirteen}_4',
                                   f'{sentinel_base_components_thirteen}_1',
                                   f'{sentinel_base_components_thirteen}_2',
                                   f'{sentinel_base_components_thirteen}_3',
                                   f'{sentinel_base_components_thirteen}_4',
                                   f'{sentinel_base_phase_thirteen}_1',
                                   f'{sentinel_base_phase_thirteen}_2',
                                   f'{sentinel_base_phase_thirteen}_3',
                                   f'{sentinel_base_phase_thirteen}_4',
                                   'lat',
                                   'lon',
                                   'latitude_longitude'}

    expected_sentinel_results_thirteen = {'authority': None,
                                          'cs': None,
                                         'gcs': None,
                                         'gcs_epsg': None,
                                         'n_bands': 19,
                                         'proj_cs': None,
                                         'proj_epsg': None,
                                         'reference_image': 'reference_images/sentinel_reference_seven.nc',  # Should be same output as test case 7.
                                         'spatial_extent': [32.95, 33.09, -115.45, -115.15],
                                         'variables': sentinel_variables_seven,
                                         'width': 360,
                                         'height': 168}
    
    check_request_output(sentinel_output_thirteen[0], expected_sentinel_results_thirteen)
    clean_test_artefacts(sentinel_directory_thirteen)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')
"""

## UAVSAR POLSAR Pauli

The tests below will use the UAVSAR POLSAR Pauli collection (for UAT this is mirrored in the EEDTEST CMR environment).

The UAVSAR POLSAR data consists of 3 variables of data where each band represents a science variable (e.g., "HH minus VV, Red Band", "HV,Green Band" and "HH plus VV, Blue Band").

### UAVSAR test case 1:

This test combines a bounding box spatial subset with a variable subset (`Band1`).



In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1422449017-ASF',
                Environment.UAT: 'G1244144733-EEDTEST',
                Environment.SIT: 'G1244144733-EEDTEST'}

    uavsar_directory_one = 'uavsar_test_one'
    
    uavsar_request_one = Request(collection=environment_information['uavsar_collection'],
                                 spatial=BBox(-145.75, 64.0, -145.70, 64.1),
                                 variables=['Band1'], format='image/tiff',
                                 granule_id=granules[environment_information['env']])

    uavsar_output_one = make_request_and_download_result(harmony_client, uavsar_request_one,
                                                         uavsar_directory_one)

    uavsar_variable_base_one = 'djNEON_00101_17095_012_170910_L090_CX_01_pauli_Band1_subsetted__Band1__subsetted_band'
    

    uavsar_variables_one = {f'{uavsar_variable_base_one}_1'}

    expected_uavsar_results_one = {'authority': 'EPSG',
                                   'cs': 'Geographic',
                                   'gcs': 'WGS 84',
                                   'gcs_epsg': '4326',
                                   'n_bands': 1,
                                   'proj_cs': None,
                                   'proj_epsg': None,
                                   'reference_image': 'reference_images/uavsar_reference_one.tiff',
                                   'spatial_extent': [64.0, 64.1, -145.75, -145.7],
                                   'variables': uavsar_variables_one,
                                   'width': 450, 
                                   'height': 1800}
    
    check_request_output(uavsar_output_one[0], expected_uavsar_results_one)
    clean_test_artefacts(uavsar_directory_one)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 2:

This test combines a bounding box spatial subset with a variable subset (`Band2`).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1422449017-ASF',
                Environment.UAT: 'G1244144733-EEDTEST',
                Environment.SIT: 'G1244144733-EEDTEST'}

    uavsar_directory_two = 'uavsar_test_two'
    
    uavsar_request_two = Request(collection=environment_information['uavsar_collection'],
                                 spatial=BBox(-145.9, 63.75, -145.8, 63.80),
                                 variables=['Band2'], format='image/tiff',
                                 granule_id=granules[environment_information['env']])

    uavsar_output_two = make_request_and_download_result(harmony_client, uavsar_request_two,
                                                         uavsar_directory_two)

    uavsar_variables_two = {'djNEON_00101_17095_012_170910_L090_CX_01_pauli_Band2_subsetted__Band2__subsetted_band_1'}

    expected_uavsar_results_two = {'authority': 'EPSG',
                                   'cs': 'Geographic',
                                   'gcs': 'WGS 84',
                                   'gcs_epsg': '4326',
                                   'n_bands': 1,
                                   'proj_cs': None,
                                   'proj_epsg': None,
                                   'reference_image': 'reference_images/uavsar_reference_two.tiff',
                                   'spatial_extent': [63.75, 63.80, -145.9, -145.8],
                                   'variables': uavsar_variables_two,
                                   'width': 900, 
                                   'height': 900}
    
    check_request_output(uavsar_output_two[0], expected_uavsar_results_two)
    clean_test_artefacts(uavsar_directory_two)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 3:

This test combines a bounding box spatial subset with a variable subset (`Band3`).

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV").

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1422449017-ASF',
                Environment.UAT: 'G1244144733-EEDTEST',
                Environment.SIT: 'G1244144733-EEDTEST'}

    uavsar_directory_three = 'uavsar_test_three'
    
    uavsar_request_three = Request(collection=environment_information['uavsar_collection'],
                                   spatial=BBox(-145.85, 63.85, -145.80, 63.90),
                                   variables=['Band3'], format='image/tiff',
                                   granule_id=granules[environment_information['env']])

    uavsar_output_three = make_request_and_download_result(harmony_client, uavsar_request_three,
                                                           uavsar_directory_three)

    uavsar_variables_three = {'djNEON_00101_17095_012_170910_L090_CX_01_pauli_Band3_subsetted__Band3__subsetted_band_1'}

    expected_uavsar_results_three = {'authority': 'EPSG',
                                     'cs': 'Geographic',
                                     'gcs': 'WGS 84',
                                     'gcs_epsg': '4326',
                                     'n_bands': 1,
                                     'proj_cs': None,
                                     'proj_epsg': None,
                                     'reference_image': 'reference_images/uavsar_reference_three.tiff',
                                     'spatial_extent': [63.85, 63.90, -145.85, -145.80],
                                     'variables': uavsar_variables_three,
                                     'width': 450, 
                                     'height': 900}
    
    check_request_output(uavsar_output_three[0], expected_uavsar_results_three)
    clean_test_artefacts(uavsar_directory_three)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 4:

This test combines a bounding box spatial subset with a variable subset for multiple variables (`Band1` and `Band2`).

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV") this test requests 2 bands. 

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1366089385-ASF',
                Environment.UAT: 'G1244144737-EEDTEST',
                Environment.SIT: 'G1244144737-EEDTEST'}

    uavsar_directory_four = 'uavsar_test_four'
    
    uavsar_request_four = Request(collection=environment_information['uavsar_collection'],
                                  spatial=BBox(-123.39, 40.05, -123.34, 40.06),
                                  variables=['Band1', 'Band2'], format='image/tiff',
                                  granule_id=granules[environment_information['env']])

    uavsar_output_four = make_request_and_download_result(harmony_client, uavsar_request_four,
                                                          uavsar_directory_four)

    uavsar_variables_four = {'eelriv_06508_16103_002_161206_L090_CX_02_pauli_subsetted__Band1__subsetted_band_1',
                             'eelriv_06508_16103_002_161206_L090_CX_02_pauli_subsetted__Band2__subsetted_band_1'}

    expected_uavsar_results_four = {'authority': 'EPSG',
                                    'cs': 'Geographic',
                                    'gcs': 'WGS 84',
                                    'gcs_epsg': '4326',
                                    'n_bands': 2,
                                    'proj_cs': None,
                                    'proj_epsg': None,
                                    'reference_image': 'reference_images/uavsar_reference_four.tiff',
                                    'spatial_extent': [40.05, 40.06, -123.39, -123.34],
                                    'variables': uavsar_variables_four,
                                    'width': 900, 
                                    'height': 180}
    
    check_request_output(uavsar_output_four[0], expected_uavsar_results_four)
    clean_test_artefacts(uavsar_directory_four)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 5:

This test performs a bounding box spatial subset, while requesting all variables.

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., \"HH - VV\"), so the output is expected to have a total of 3 bands (1 each for `Band1`, `Band2` and `Band3`).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1366089385-ASF',
                Environment.UAT: 'G1244144737-EEDTEST',
                Environment.SIT: 'G1244144737-EEDTEST'}

    uavsar_directory_five = 'uavsar_test_five'
    
    uavsar_request_five = Request(collection=environment_information['uavsar_collection'],
                                  spatial=BBox(-123.69, 39.95, -123.59, 40.0),
                                  format='image/tiff',
                                  granule_id=granules[environment_information['env']])

    uavsar_output_five = make_request_and_download_result(harmony_client, uavsar_request_five,
                                                          uavsar_directory_five)

    uavsar_variables_five = {'eelriv_06508_16103_002_161206_L090_CX_02_pauli_subsetted__all__subsetted_band_1',
                             'eelriv_06508_16103_002_161206_L090_CX_02_pauli_subsetted__all__subsetted_band_2', 
                             'eelriv_06508_16103_002_161206_L090_CX_02_pauli_subsetted__all__subsetted_band_3',}

    expected_uavsar_results_five = {'authority': 'EPSG',
                                    'cs': 'Geographic',
                                    'gcs': 'WGS 84',
                                    'gcs_epsg': '4326',
                                    'n_bands': 3,
                                    'proj_cs': None,
                                    'proj_epsg': None,
                                    'reference_image': 'reference_images/uavsar_reference_five.tiff',
                                    'spatial_extent': [39.95, 40.0, -123.69, -123.59],
                                    'variables': uavsar_variables_five,
                                    'width': 1800, 
                                    'height': 900}
    
    check_request_output(uavsar_output_five[0], expected_uavsar_results_five)
    clean_test_artefacts(uavsar_directory_five)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 6:

This test performs a bounding box spatial subset that crosses the Equator, while explicitly requesting all variables by identifier "all".

Each band in the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV") so the output is expected to have 3 bands (1 each for `Band1`, `Band2` and `Band3`).

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1232482059-ASF',
                Environment.UAT: 'G1244144732-EEDTEST',
                Environment.SIT: 'G1244144732-EEDTEST'}

    uavsar_directory_six = 'uavsar_test_six'
    
    uavsar_request_six = Request(collection=environment_information['uavsar_collection'],
                                 variables=['all'],
                                 spatial=BBox(11.57, -0.03, 11.60, 0.02),
                                 format='image/tiff',
                                 granule_id=granules[environment_information['env']])

    uavsar_output_six = make_request_and_download_result(harmony_client, uavsar_request_six,
                                                         uavsar_directory_six)

    uavsar_base_six = 'eighty_14047_16008_006_160225_L090_CX_01_pauli_subsetted__all__subsetted'

    uavsar_variables_six = {f'{uavsar_base_six}_band_1',
                            f'{uavsar_base_six}_band_2',
                            f'{uavsar_base_six}_band_3'}

    expected_uavsar_results_six = {'authority': 'EPSG',
                                   'cs': 'Geographic',
                                   'gcs': 'WGS 84',
                                   'gcs_epsg': '4326',
                                   'n_bands': 3,
                                   'proj_cs': None,
                                   'proj_epsg': None,
                                   'reference_image': 'reference_images/uavsar_reference_six.tiff',
                                   'spatial_extent': [-0.03, 0.02, 11.57, 11.60],
                                   'variables': uavsar_variables_six,
                                   'width': 540, 
                                   'height': 900}
    
    check_request_output(uavsar_output_six[0], expected_uavsar_results_six)
    clean_test_artefacts(uavsar_directory_six)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 7:

This test performs a bounding box spatial subset in the Northern Hemisphere, explicitly requesting a bounding box that is larger than the input granule extent. To limit the size of the reference image stored in the repository, the request specifies a variable subset for only `Band1`, and the output dimensions are specified to be coarser resolution than the input granule.

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV") and has four colour bands, so the output is expected to have a total of 4 bands.

**Note, this test used to specify "all" for variables.**

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1402334529-ASF',
                Environment.UAT: 'G1244144730-EEDTEST',
                Environment.SIT: 'G1244144730-EEDTEST'}

    uavsar_directory_seven = 'uavsar_test_seven'
    
    uavsar_request_seven = Request(collection=environment_information['uavsar_collection'],
                                   variables=['Band1'], spatial=BBox(-139, 61, -137, 62),
                                   format='image/tiff', height=870, width=1129,
                                   granule_id=granules[environment_information['env']])

    uavsar_output_seven = make_request_and_download_result(harmony_client, uavsar_request_seven,
                                                           uavsar_directory_seven)

    uavsar_band1_base_seven = ('kluane_01902_17064_005_170615_L090_CX_01_pauli_Band1_'
                               'regridded_subsetted__Band1__colored_band')

    uavsar_variables_seven = {f'{uavsar_band1_base_seven}_1',
                              f'{uavsar_band1_base_seven}_2',
                              f'{uavsar_band1_base_seven}_3',
                              f'{uavsar_band1_base_seven}_4'}

    expected_uavsar_results_seven = {'authority': 'EPSG',
                                     'cs': 'Geographic',
                                     'gcs': 'WGS 84',
                                     'gcs_epsg': '4326',
                                     'n_bands': 4,
                                     'proj_cs': None,
                                     'proj_epsg': None,
                                     'reference_image': 'reference_images/uavsar_reference_seven.tiff',
                                     'spatial_extent': [61.57, 61.76, -138.36, -137.85],
                                     'variables': uavsar_variables_seven,
                                     'width': 1129, 
                                     'height': 870}
    
    check_request_output(uavsar_output_seven[0], expected_uavsar_results_seven)
    clean_test_artefacts(uavsar_directory_seven)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 8:

This test combines a bounding box spatial subset with a variable subset (`Band2`). The bounding box extends past the western bound of the input granule extent.

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV") and has four colour bands, so the output is expected to have a total of 4 bands.

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1223713114-ASF',
                Environment.UAT: 'G1244144731-EEDTEST',
                Environment.SIT: 'G1244144731-EEDTEST'}

    uavsar_directory_eight = 'uavsar_test_eight'
    
    uavsar_request_eight = Request(collection=environment_information['uavsar_collection'],
                                   spatial=BBox(-72.3, -40.02, -72.0, -40.0),
                                   variables=['Band2'], format='image/tiff',
                                   granule_id=granules[environment_information['env']])

    uavsar_output_eight = make_request_and_download_result(harmony_client, uavsar_request_eight,
                                                           uavsar_directory_eight)

    uavsar_variable_base_eight = ('ChiVol_00700_15030_010_150330_L090_CX_01_pauli_Band2_'
                                  'subsetted__Band2__colored_band')

    uavsar_variables_eight = {f'{uavsar_variable_base_eight}_1',
                              f'{uavsar_variable_base_eight}_2',
                              f'{uavsar_variable_base_eight}_3',
                              f'{uavsar_variable_base_eight}_4'}

    expected_uavsar_results_eight = {'authority': 'EPSG',
                                     'cs': 'Geographic',
                                     'gcs': 'WGS 84',
                                     'gcs_epsg': '4326',
                                     'n_bands': 4,
                                     'proj_cs': None,
                                     'proj_epsg': None,
                                     'reference_image': 'reference_images/uavsar_reference_eight.tiff',
                                     'spatial_extent': [-40.02, -40.0, -72.17, -72.0],
                                     'variables': uavsar_variables_eight,
                                     'width': 3004, 
                                     'height': 360}
    
    check_request_output(uavsar_output_eight[0], expected_uavsar_results_eight)
    clean_test_artefacts(uavsar_directory_eight)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 9:

This test combines a bounding box spatial subset with a temporal subset. All three variables are explicitly listed in the request. The temporal subset should return only a single granule, and apply the spatial and variable subset to that granule.

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV") and has four colour bands, so the output is expected to have a total of 12 bands (4 each for `Band1`, `Band2` and `Band3`).

**Note, this test used to specify "all" for variables.**

In [ ]:
if environment_information is not None:
    uavsar_directory_nine = 'uavsar_test_nine'
    
    uavsar_request_nine = Request(collection=environment_information['uavsar_collection'],
                                  variables=['Band1', 'Band2', 'Band3'],
                                  spatial=BBox(-89.02, 28.98, -89.00, 29.00),
                                  temporal={'start': datetime(2016, 11, 17, 23, 27, 0),
                                            'stop': datetime(2016, 11, 17, 23, 28, 0)},
                                  format='image/tiff')

    uavsar_output_nine = make_request_and_download_result(harmony_client, uavsar_request_nine,
                                                          uavsar_directory_nine)

    uavsar_band1_base_nine = 'gulfco_27086_16101_004_161117_L090_CX_01_pauli_subsetted__Band1__colored_band'
    uavsar_band2_base_nine = 'gulfco_27086_16101_004_161117_L090_CX_01_pauli_subsetted__Band2__colored_band'
    uavsar_band3_base_nine = 'gulfco_27086_16101_004_161117_L090_CX_01_pauli_subsetted__Band3__colored_band'

    uavsar_variables_nine = {f'{uavsar_band1_base_nine}_1',
                             f'{uavsar_band1_base_nine}_2',
                             f'{uavsar_band1_base_nine}_3',
                             f'{uavsar_band1_base_nine}_4',
                             f'{uavsar_band2_base_nine}_1',
                             f'{uavsar_band2_base_nine}_2',
                             f'{uavsar_band2_base_nine}_3',
                             f'{uavsar_band2_base_nine}_4',
                             f'{uavsar_band3_base_nine}_1',
                             f'{uavsar_band3_base_nine}_2',
                             f'{uavsar_band3_base_nine}_3',
                             f'{uavsar_band3_base_nine}_4'}

    expected_uavsar_results_nine = {'authority': 'EPSG',
                                    'cs': 'Geographic',
                                    'gcs': 'WGS 84',
                                    'gcs_epsg': '4326',
                                    'n_bands': 12,
                                    'proj_cs': None,
                                    'proj_epsg': None,
                                    'reference_image': 'reference_images/uavsar_reference_nine.tiff',
                                    'spatial_extent': [28.98, 29.00, -89.02, -89.00],
                                    'variables': uavsar_variables_nine,
                                    'width': 360, 
                                    'height': 360}
    
    check_request_output(uavsar_output_nine[0], expected_uavsar_results_nine)
    clean_test_artefacts(uavsar_directory_nine)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 10:

This test combines a bounding box spatial subset with a variable subset (`Band1`). The requested output will be a `.png` format file.

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV") and has four colour bands, so the output is expected to have a total of 4 bands.

In [ ]:
if environment_information is not None:
    granules = {Environment.PROD: 'G1422449017-ASF',
                Environment.UAT: 'G1244144733-EEDTEST',
                Environment.SIT: 'G1244144733-EEDTEST'}

    uavsar_directory_ten = 'uavsar_test_ten'
    
    uavsar_request_ten = Request(collection=environment_information['uavsar_collection'],
                                 variables=['Band1'], spatial=BBox(-145.75, 64.0, -145.70, 64.1),
                                 granule_id=granules[environment_information['env']],
                                 format='image/png')

    uavsar_output_ten = make_request_and_download_result(harmony_client, uavsar_request_ten,
                                                         uavsar_directory_ten)

    uavsar_variables_ten = {'Band1', 'Band2', 'Band3', 'Band4'}

    expected_uavsar_results_ten = {'authority': None,
                                   'cs': None,
                                   'file_type': 'png',
                                   'gcs': None,
                                   'gcs_epsg': None,
                                   'n_bands': 4,
                                   'proj_cs': None,
                                   'proj_epsg': None,
                                   'reference_image': 'reference_images/uavsar_reference_ten.png',
                                   'spatial_extent': [64.0, 64.1, -145.75, -145.70],
                                   'variables': uavsar_variables_ten,
                                   'width': 450, 
                                   'height': 1800}
    
    check_request_output(uavsar_output_ten[0], expected_uavsar_results_ten)
    clean_test_artefacts(uavsar_directory_ten)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')

### UAVSAR test case 11:

This test combines a bounding box spatial subset with a variable subset (`Band1`). The requested output will be a `.gif` format file.

Each input variable for the UAVSAR POLSAR Pauli collection is a science variable (e.g., "HH - VV") and has four colour bands.

# TO FIX: Request gets `gdal_translate` error:

```
2022-04-08T17:24:25.825Z [debug]: INFO:root:gdal_translate '-of' 'GIF' '-scale' '/tmp/tmpke2epqg1/result.tif' '/tmp/tmpke2epqg1/translated.gif'
2022-04-08T17:24:25.911Z [debug]: ERROR 6: GIF driver only supports one band images.
```

In [ ]:
"""
if environment_information is not None:
    granules = {Environment.PROD: 'G1422449017-ASF',
                Environment.UAT: 'G1244144733-EEDTEST',
                Environment.SIT: 'G1244144733-EEDTEST'}

    uavsar_directory_eleven = 'uavsar_test_eleven'
    
    uavsar_request_eleven = Request(collection=environment_information['uavsar_collection'],
                                    variables=['Band1'], spatial=BBox(-145.75, 64.0, -145.70, 64.1),
                                    granule_id=granules[environment_information['env']],
                                    format='image/gif')

    uavsar_output_eleven = make_request_and_download_result(harmony_client, uavsar_request_eleven,
                                                            uavsar_directory_eleven)

    uavsar_variables_eleven = {'Band1', 'Band2', 'Band3', 'Band4'}

    expected_uavsar_results_eleven = {'authority': None,
                                      'cs': None,
                                      'file_type': 'gif',
                                      'gcs': None,
                                      'gcs_epsg': None,
                                      'n_bands': 4,
                                      'proj_cs': None,
                                      'proj_epsg': None,
                                      # 'reference_image': 'reference_images/uavsar_query11_reference.tiff',
                                      'spatial_extent': [64.0, 64.1, -145.75, -145.70],
                                      'variables': uavsar_variables_eleven,
                                      'width': 450, 
                                      'height': 1800}
    
    check_request_output(uavsar_output_eleven[0], expected_uavsar_results_eleven)
    clean_test_artefacts(uavsar_directory_eleven)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')
"""

## GHRSST Level 4 MUR Global Foundation Sea Surface Temperature Analysis

The tests below will use the GHRSST Level 4 MUR Global Foundation Sea Surface Temperature Analysis from PO.DAAC.

### MUR Test Case 1:

This test combines a resampled output with a variable subset (`analysed_sst`) format will be a `.png`.


In [ ]:
  if environment_information is not None:
    granules = { 
        Environment.UAT: 'G1240564919-POCLOUD',
        Environment.LOCAL: 'G1240564919-POCLOUD'        
    }

    mur_directory_one = 'mur_test_one'

    mur_request_one = Request(collection=environment_information['mur_collection'],
                              variables=['analysed_sst'],
                              granule_id=granules[environment_information['env']],
                              format='image/png',
                              width=900, height=450)

    mur_output_one = make_request_and_download_result(
        harmony_client, mur_request_one, mur_directory_one
    )

    mur_variables_one = {'Band1', 'Band2', 'Band3', 'Band4'}

    expected_mur_results_one = {'authority': None,
                                'cs': None,
                                'file_type': 'png',
                                'gcs': None,
                                'gcs_epsg': None,
                                'n_bands': 4,
                                'proj_cs': None,
                                'proj_epsg': None,
                                'reference_image': 'reference_images/mur_reference_one.png',
                                'spatial_extent': None,
                                'variables': mur_variables_one,
                                'width': 900,
                                'height': 450}
    
    check_request_output(mur_output_one[0], expected_mur_results_one)
    clean_test_artefacts(mur_directory_one)
else:
    print('Skipping test: HGA regression tests not configured for this environment.')